In [25]:
from ls_helper.new_models import get_p_access, platforms_overview, ProjectData

twitter = "twitter"
en = "en"
tw_en = {"platform": twitter, "language": en}
_default = tw_en

po: ProjectData = platforms_overview.get_project(get_p_access(**_default))

In [26]:
aggree = po.get_agreement_data()

In [27]:
single_vis = aggree.agreement_metrics.single_choice.variables[
    "nep_materiality_visual"
]

In [28]:
single_vis.model_dump(exclude={"conflicts"})

{'kappa': 0.5884,
 'gwet': 0.8063,
 'percent_agreement': 0.8803,
 'total_rows': 1295,
 'agreement_count': 1140,
 'disagreement_count': 155,
 'counts': {'material': 348, 'immaterial': 254, 'NONE': 1684},
 'option_results': {'material': {'error': None,
   'kappa': -0.1666,
   'gwet': 0.5296,
   'percent_agreement': 0.6047,
   'number': 215,
   'agreement_count': 130,
   'disagreement_count': 85,
   'total_selections': 348,
   'total_rows_selected': None,
   'total_tasks_analyzed': None,
   'all_present_count': None,
   'all_absent_count': None,
   'conflicts': None},
  'immaterial': {'error': None,
   'kappa': -0.2808,
   'gwet': 0.2958,
   'percent_agreement': 0.4457,
   'number': 175,
   'agreement_count': 78,
   'disagreement_count': 97,
   'total_selections': 254,
   'total_rows_selected': None,
   'total_tasks_analyzed': None,
   'all_present_count': None,
   'all_absent_count': None,
   'conflicts': None},
  'NONE': {'error': None,
   'kappa': -0.4695,
   'gwet': 0.8183,
   'percen

In [29]:
mp = po.get_raw_annotations_results(accepted_ann_age=20)
rdf_ = mp.raw_annotation_df

Get recent, gets latest annotation


In [30]:
rdf = rdf_.copy()

In [31]:
multi_annotator_tasks = rdf.groupby("task_id")["ann_id"].nunique()
multi_annotator_tasks = multi_annotator_tasks[multi_annotator_tasks > 1].index

# Filter raw dataframe
filtered_df = rdf[rdf["task_id"].isin(multi_annotator_tasks)]

# Filter for nep_materiality_visual with all indices
nep_mv_filtered = filtered_df[
    filtered_df["category"].str.match(r"nep_materiality_visual_\d+")
]

# Count total rows after filtering
print(
    f"Total nep_materiality_visual rows after filtering: {len(nep_mv_filtered)}"
)

# Extract and count options (assuming values are stored as lists)
# This handles values stored as lists ['option']
nep_mv_filtered["option"] = nep_mv_filtered["value"].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x
)

# Count by option
option_counts = nep_mv_filtered["option"].value_counts()
print(f"Option counts:\n{option_counts}")

# Sum of material and immaterial options
if "material" in option_counts and "immaterial" in option_counts:
    total = option_counts["material"] + option_counts["immaterial"]
    print(f"Sum of material + immaterial: {total}")

Total nep_materiality_visual rows after filtering: 801
Option counts:
option
material      462
immaterial    339
Name: count, dtype: int64
Sum of material + immaterial: 801


/tmp/ipykernel_21878/1333088454.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nep_mv_filtered['option'] = nep_mv_filtered['value'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)


In [33]:
nep_mv = rdf[
    rdf["category"].isin([f"nep_materiality_visual_{i}" for i in range(0, 4)])
]
len(nep_mv)

890

In [20]:
len(nep_mv)

890

In [34]:
(
    single_vis.option_results["material"].total_selections
    + single_vis.option_results["immaterial"].total_selections
)

602